In [ ]:
from naive_bayes_classifier import *

In [ ]:
# Convert csv file to a pandas DataFrame

df = pd.read_csv('inflammation_diagnosis.csv', sep=';')

df.columns = df.columns.str.strip()

print(df)

In [ ]:
# Create 'disease' column and drop the other two columns

df['inflammation'] = df['inflammation'].map({'yes': True, 'no': False})
df['nephritis'] = df['nephritis'].map({'yes': True, 'no': False})

df['disease'] = df['inflammation'] | df['nephritis']

df = df.drop(columns=['inflammation', 'nephritis'])

df_complete = df

df_complete = df_complete.drop(columns=['disease'])

print(df_complete)

In [ ]:
# Create 'continuous' array

continuous = []

for column in df.columns[:-1]: # Iterate over every column except the label column
    first_value = df[column].iloc[0]
    if first_value == 'yes' or first_value == 'no': # If discrete value
        continuous.append(False)
    else: # If continuous value
        continuous.append(True)
        
print(continuous)

In [ ]:
# Randomize the dataframe's sample and split it into train and test set

shuffle_df = df.sample(frac=1, random_state=42).reset_index(drop=True)

train_size = int(0.8 * len(shuffle_df))

train_df = shuffle_df.iloc[:train_size]
test_df = shuffle_df.iloc[train_size:]

print(train_df)
print(test_df)

In [ ]:
# Instantiate the NaiveBayes class

naive_bayes = NaiveBayes(continuous)

In [ ]:
target_name = "disease"
naive_bayes.fit(train_df, target_name)

In [ ]:
print(naive_bayes.priors)

In [ ]:
print(naive_bayes.likelihoods)

In [ ]:
print(naive_bayes.gaussian_parameters)

In [ ]:
naive_bayes.predict_probability(df_complete)